In [1]:
# Reconocimiento de ip´s en la red
import os
import threading

info= []
def search(ip_address, info):
    # enviar ping
    command= "ping -c 4 " + ip_address + ' | grep "4 packets"' + " | awk '{print $4,$5}'"
    response= os.popen(command).read()
    if "4 received" in response:
        info.append(ip_address)

for ip in range (1,255):
    current_ip= "192.168.100." + str(ip) #range ip

    run=threading.Thread(target=search, args=(current_ip, info))
    run.start()

In [6]:
# Conectar a postgreSQL
import psycopg2
from psycopg2 import Error

try:
    conexion = psycopg2.connect(host="localhost",
                                user="home", 
                                password="1234", 
                                database="ip_recog")
    boolean= 1
    print('>>> CONEXIÓN EXITOSA')
except:
    print("Error: Configuración no válida")
    boolean= 0
    
# Establecer conexión automática
if boolean==1:
    conexion.autocommit= True
else:
    print("No existe conexión con postgre")

>>> CONEXIÓN EXITOSA


In [3]:
def extraer_datos():
    cursor= conexion.cursor()
    try:
        query= """SELECT * FROM ip_found;""" 
        cursor.execute(query)
        data= cursor.fetchall()
        print(f">>> {query}")
        print(data)
        cursor.close()
    except:
        print(f">>> {query}")
        print("Error: Tabla no existente")
        
def enviar_data(ip,info):
    cursor= conexion.cursor()
    try:
        query= f"""INSERT INTO ip_found (ip) VALUES ('{ip}')\n"""
        cursor.execute(query)
        cursor.close()
        print(f">>> {query}")
    except:
        print("Error: ")
        print(query)

In [7]:
print(f""">>> Data send postgre <<<\n{info}\n""")

>>> Data send postgre <<<
['192.168.100.5', '192.168.100.1', '192.168.100.7', '192.168.100.30', '192.168.100.6']



In [16]:
# extraer_datos()
try:
    extraer_datos()
    print("\n")
    # enviar_data()
    indice = 0
    longitud = len(info)
    longitud = longitud - 1

    if boolean==1:
        ContinueExecuting = False ###### Write True for send data ######
        while ContinueExecuting:
            for i in info:
                ip= i
                run2=threading.Thread(target=enviar_data, args=(ip, info))
                run2.start()
                ContinueExecuting = False
    else:
        print("""Error conexión: Imposible enviar datos a tabla postgre""")
except:
    print("Error conexión: Imposible extraer datos de tabla postgre")

>>> SELECT * FROM ip_found;
[('192.168.100.88',), ('192.168.100.5',), ('192.168.100.1',), ('192.168.100.7',), ('192.168.100.30',), ('192.168.100.6',), ('192.168.100.5',), ('192.168.100.1',), ('192.168.100.7',), ('192.168.100.30',), ('192.168.100.6',)]


